In [1]:
from pathlib import Path
import pandas as pd


thispath = Path.cwd().resolve()

datadir = Path(thispath.parent / "data")

labels = pd.read_csv(Path(datadir / "labels.csv"), index_col='image_num')

count_scc = (labels['cancer_scc'] == 1).sum()
count_nscc_adeno = (labels['cancer_nscc_adeno'] == 1).sum()
count_nscc_squamous = (labels['cancer_nscc_squamous'] == 1).sum()
count_no_cancer = (labels['no_cancer'] == 1).sum()

print(f"Nº scc: {count_scc}")
print(f"Nº nscc_adeno: {count_nscc_adeno}")
print(f"Nº nscc_squamous: {count_nscc_squamous}")
print(f"Nº no_cancer: {count_no_cancer}")

wsi_scc = labels.index[labels['cancer_scc'] == 1].tolist()
wsi_nscc_adeno = labels.index[labels['cancer_nscc_adeno'] == 1].tolist()
wsi_nscc_squamous = labels.index[labels['cancer_nscc_squamous'] == 1].tolist()
wsi_no_cancer = labels.index[labels['no_cancer'] == 1].tolist()

scc_nscc_adeno = []
scc_nscc_squamous = []
scc_no_cancer = []
nscc_adeno_squamous = []
scc_nscc_adeno_squamous = []
nscc_adeno_no_cancer = []
nscc_squamous_no_cancer = []


for scc in wsi_scc:
    
    for nscc_adeno in wsi_nscc_adeno:
        if scc == nscc_adeno and nscc_adeno not in scc_nscc_adeno:
            scc_nscc_adeno.append(nscc_adeno)

        for nscc_squamous in wsi_nscc_squamous:
            if scc == nscc_squamous and nscc_squamous not in scc_nscc_squamous:
                scc_nscc_squamous.append(nscc_squamous)

            if nscc_adeno == nscc_squamous and nscc_squamous not in nscc_adeno_squamous:
                nscc_adeno_squamous.append(nscc_squamous)
            
            if scc == nscc_adeno == nscc_squamous and nscc_squamous not in scc_nscc_adeno_squamous:
                scc_nscc_adeno_squamous.append(nscc_squamous)


print("WSI with more than one label")
print(f"SCC and Adeno: {len(scc_nscc_adeno)}")
print(f"SCC and Squamous: {len(scc_nscc_squamous)}")
print(f"Adeno and Squamous: {len(nscc_adeno_squamous)}")
print(f"SCC, Adeno and Squamous: {len(scc_nscc_adeno_squamous)}")

Nº scc: 232
Nº nscc_adeno: 690
Nº nscc_squamous: 251
Nº no_cancer: 339
WSI with more than one label
SCC and Adeno: 13
SCC and Squamous: 180
Adeno and Squamous: 12
SCC, Adeno and Squamous: 11


Check the splits

In [36]:
from ast import literal_eval

k = 10
fold = 0

data_split = pd.read_csv(Path(datadir / f"{k}_fold_crossvalidation_data_split.csv"), index_col=0)

# Data fold 0
train_dataset = literal_eval(data_split.loc[fold]["images_train"])
test_dataset = literal_eval(data_split.loc[fold]["images_test"])
train_labels = literal_eval(data_split.loc[fold]["labels_train"])
test_labels = literal_eval(data_split.loc[fold]["labels_test"])
print(train_dataset)
print(type(train_dataset))
print(test_dataset)
print(len(test_dataset))
print(train_labels)
print(len(train_labels))
print(test_labels)
print(type(test_labels))


['000029269500258388', '000029488200270022', '000029488200270028', '000029488200270029', '000029496800270408', '000029496800270409', '000029496800270412', '000029496800270413', '000029496800270415', '000029496800270416', '000029496800270417', '000029496800270418', '000029536100271824', '000029709100281888', '000029832000288268', '000029919100292999', '000030069800299917', '000030274100310453', '000030303300314205', '000030303300314206', '000030303300314209', '000030303300314210', '000030397000318685', '000030397000318686', '000030397000318689', '000030399700317760', '000030399700317761', '000030399700317762', '000030399700317763', '000030412700319317', '000030412700319318', '000030412700319319', '000030412700319320', '000030412700319321', '000030412700319322', '000030438500320102', '000030438500320110', '000030438500320111', '000030438500320112', '000030443200320380', '000030466600322332', '000030467500322347', '000030473100323233', '000030487200323364', '000030487200323365', '00003048

In [ ]:
# Load features from MoCo model
experiment_name = "MoCo_try_Adam"

mocodir = Path(thispath.parent / 
               "trained_models" / 
               "MoCo" / 
               experiment_name)

df_features = pd.read_csv(mocodir / f"features_{experiment_name}.csv", index_col=0)


In [54]:
from natsort import natsorted
from tqdm import tqdm

# Train and validation
pyhistdir = Path(datadir / "Mask_PyHIST_v2")

dataset_path = natsorted([i for i in pyhistdir.rglob("*_densely_filtered_paths.csv") if "LungAOEC" in str(i)])

path_patches = {}
patches_names = {}
for wsi_patches in tqdm(dataset_path, desc="Selecting patches to extract features"):

    csv_instances = pd.read_csv(wsi_patches).to_numpy()
    
    name = wsi_patches.parent.stem
    path_patches[name] = csv_instances
    patches_names[name] = []

    for instance in csv_instances:
            patches_names[name].append(str(instance).split("/")[-1])

Selecting patches to extract features: 100%|██████████| 1318/1318 [00:49<00:00, 26.45it/s]


In [55]:
patches_train = []
patches_test = []
for value, key in zip(patches_names.values(), path_patches.keys()):
        
    if key in train_dataset:
        patches_train.extend(value)
    if key in test_dataset:
        patches_test.extend(value)

print(len(patches_train))
print(len(patches_test))



1968670
474965


In [57]:
features_train = df_features.loc[patches_train]
features_valid = df_features.loc[patches_test]

In [13]:
k = 10

data_split = pd.read_csv(Path(datadir / f"{k}_fold_crossvalidation_data_split.csv"), index_col=0)

for fold, _ in data_split.iterrows():
    train_dataset = literal_eval(data_split.loc[fold]["images_train"])
    test_dataset = literal_eval(data_split.loc[fold]["images_test"])
    train_labels = literal_eval(data_split.loc[fold]["labels_train"])
    test_labels = literal_eval(data_split.loc[fold]["labels_test"])
    print(train_dataset)
    print(type((train_dataset)))
    print(test_dataset)
    print(len(test_dataset))
    print(train_labels)
    print(len(train_labels))
    print(test_labels)
    print(type(test_labels))


['000030274100310453.svs', '000029536100271824.svs', '000031016200565072.svs', '000031016200565066.svs', '000034025900540935.svs', '000034025900534170.svs', '000031016200350116.svs', '000031016200349368.svs', '000030507100323487.svs', '000029832000288268.svs', '000031411300369843.svs', '000031197100358911.svs', '000030443200320380.svs', '000030075400300796.svs', '000030069800299917.svs', '000029709100281888.svs', '000029389200263815.svs', '000035865700716616.svs', '000034819700614035.svs', '000034819700614033.svs', '000034819700614021.svs', '000034819700614011.svs', '000034819700614009.svs', '000034819700614007.svs', '000034812300613580.svs', '000034812300609243.svs', '000034819700614013.svs', '000034238100556679.svs', '000031305500363714.svs', '000033478000483965.svs', '000034939400624566.svs', '000034238100556681.svs', '000034238100556677.svs', '000034238100555881.svs', '000034238100555879.svs', '000034045000540010.svs', '000034101300540428.svs', '000034101000539241.svs', '0000337827